In [1]:
from helpers import *
import torch

In [2]:
def train_model(v_tr, s_tr, g=100, l=100, nsamp=None):
    
    nsamp = nsamp if nsamp else v_tr.size(0)//s_tr.size(0)
    
    xx = v_tr.t().mm(v_tr)
    ss = s_tr.t().mm(s_tr)
    
    #s_tr = s_tr.unsqueeze(1).repeat(1,nsamp,1).view(-1, 300)
    xs = v_tr.t().mm(s_tr)
    
    xx_inv = (xx + g * torch.eye(xx.size(0), dtype=torch.float32, device="cuda:0")).inverse()
    ss_inv = (ss + l * torch.eye(ss.size(0), dtype=torch.float32, device="cuda:0")).inverse()
    
    return xx_inv.mm(xs).mm(ss_inv)

def validate(v_te, s_te, w, k=5):
    """
    """
    out = v_te.mm(w)
    out = out.mm(s_te.t())
    return out

# Training

In [3]:
norm_sem=False
x_tr,y_tr,s_tr=load_train_set(norm_sem=norm_sem)
x_tr = torch.cat([x.mean(0, keepdim=True) for x in torch.chunk(x_tr, 993)])
w = train_model(x_tr.cuda(), s_tr.cuda())

993it [01:17, 12.76it/s] 


# Standard ZSL

In [4]:
x_te,y_te,s_te = load_test_set(norm_sem=norm_sem, generalized=False)
out = validate(x_te.cuda(), s_te.cuda(),  w)
test_accs = topk(out, y_te.cuda())
pp(test_accs)

500it [00:00, 1348.73it/s]


 13.54 | 20.44 | 25.43 | 29.24 | 32.61 


# Generalized ZSL

In [5]:
x_te, y_te, s_te = load_test_set(norm_sem=norm_sem, generalized=True)
out = validate(x_te.cuda(), s_te.cuda(), w)

1493it [00:00, 1914.94it/s]


In [6]:
msk_test  = y_te < 500
test_accs = topk(out[msk_test], y_te[msk_test].cuda())
pp(test_accs)

 2.43 | 6.16 | 9.47 | 12.66 | 15.46 


In [8]:
msk_train = y_te > 500
train_accs = topk(out[msk_train], y_te[msk_train].cuda())
pp(train_accs)

 42.98 | 57.01 | 64.74 | 69.62 | 73.06 


In [9]:
h_mean = h_scores(test_accs, train_accs)
pp(h_mean)

 4.59 | 11.12 | 16.53 | 21.42 | 25.53 
